In [1]:
#pip install BeautifulSoup4

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import csv
import os
import re
import time

In [3]:
data = {
    'name': [],
    'country': [],
    'data of birth': [],
    'height': [],
    'foot': [],
    'joined': [],
    'contract until': [],
    'club name': [],
    'market value':[]
}

# Request for transfermarkt.com
## All you need to do in this script is put an URL in varaible from league profile.

In [4]:
headers = {
  'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
}

url = "https://www.transfermarkt.pl/pko-ekstraklasa/startseite/wettbewerb/PL1"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
td_elements = soup.find_all('td', {'class': ['hauptlink no-border-links']})

In [6]:
links = []
for i in range(len(td_elements)):
    text = td_elements[i].find('a')['href']
    match = re.search(r'/(\d+)/', text)
    if match:
        number = match.group(1)
        links.append(number)

In [7]:
links

['9644',
 '238',
 '255',
 '324',
 '5689',
 '168',
 '428',
 '2300',
 '6112',
 '7146',
 '88',
 '759',
 '7154',
 '22431',
 '6110',
 '2118',
 '4000',
 '8936']

In [8]:
endpoint = "endpoint"

In [9]:
league_name = soup.find('h1').text

In [10]:
league_name = league_name.strip()

# Webscrapping club data from entire league

In [11]:
for i in range(len(links)):
    club_url = "https://www.transfermarkt.pl/ajax-amsterdam/kader/verein/" + endpoint + "/saison_id/2022/plus/1"
    new_club_url = club_url.replace(endpoint, links[i])    
    response = requests.get(new_club_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    endpoint = links[i]
    
    
    td_element = soup.find_all('td', {'class': ['zentriert']})
    for i in range(len(td_element)):
        if td_element[i].find('img', {'class': 'flaggenrahmen'}) is None:
            continue

        if len(td_element[i].find_all('img', {'class': 'flaggenrahmen'})) == 2:
            data['country'].append(td_element[i].find_all('img', {'class': 'flaggenrahmen'})[0]['title'] 
                             + " / " + td_element[i].find_all('img', {'class': 'flaggenrahmen'})[1]['title'])
        else:
            data['country'].append(td_element[i].find('img')['title'])
    
    all_tr = soup.find_all('tr', {'class': ['odd', 'even']})       
    for row in all_tr:
        all_td = row.find_all('td', recursive=False)
        data['name'].append( all_td[1].text.split('.')[0][:-1] )
        data['data of birth'].append( all_td[2].text )
        data['height'].append( all_td[4].text )
        data['foot'].append( all_td[5].text )
        data['joined'].append( all_td[6].text )
        data['contract until'].append( all_td[8].text )
        data['market value'].append( all_td[9].text )
        data['club name'].append(soup.find('h1').text.strip())



In [12]:
df = pd.DataFrame(data)

In [13]:
df

,name,country,data of birth,height,foot,joined,contract until,club name,market value
0,\n\n\n\n \n\n\n Vladan Kovacevi...,Serbia / Bośnia i Hercegowina,11 kwi 1998 (25),"1,92m",prawonożny,1 lip 2021,30 cze 2026,Raków Częstochowa,"8,00 mln €"
1,\n\n\n\n \n\n\n Kacper Trelowsk...,Polska,19 sie 2003 (19),"1,93m",obunożny,23 sie 2019,30 cze 2025,Raków Częstochowa,700 tys. €
2,\n\n\n\n \n\n\n Jakub Rajczykow...,Polska,4 lut 2005 (18),-,,21 lip 2022,30 cze 2025,Raków Częstochowa,25 tys. €
3,\n\n\n\n \n\n\n Stratos Svarnas...,Grecja,11 lis 1997 (25),"1,86m",prawonożny,1 lip 2022,30 cze 2023,Raków Częstochowa,"2,00 mln €"
4,\n\n\n\n \n\n\n Zoran Arsenic ...,Chorwacja,2 cze 1994 (29),"1,88m",prawonożny,1 lip 2021,30 cze 2026,Raków Częstochowa,"1,80 mln €"
...,...,...,...,...,...,...,...,...,...
546,\n\n\n\n \n\n\n Igor Lewandowsk...,Polska,9 paź 2001 (21),"1,76m",,28 wrz 2020,-,Miedź Legnica,25 tys. €
547,\n\n\n\n \n\n\n Blazej Szczepan...,Polska,20 mar 2001 (22),"1,77m",prawonożny,28 paź 2022,30 cze 2024,Miedź Legnica,-
548,\n\n\n\n \n\n\n Ángelo Henríque...,Chile,13 kwi 1994 (29),"1,77m",prawonożny,1 lip 2022,30 cze 2025,Miedź Legnica,350 tys. €
549,\n\n\n\n \n\n\n Wiktor Bogacz ...,Polska,14 lip 2004 (18),"1,93m",obunożny,6 kwi 2023,31 gru 2025,Miedź Legnica,25 tys. €


In [14]:
type(data)

dict

In [15]:
data_keys = data.keys()

In [16]:
data_keys

dict_keys(['name', 'country', 'data of birth', 'height', 'foot', 'joined', 'contract until', 'club name', 'market value'])

# Converting dictionary data type to ndarray

In [17]:
new_2d_arr = np.array([data[i] for i in data_keys])

In [18]:
new_2d_arr[:,0]

array(['\n\n\n\n \n\n\n                Vladan Kovacevic            \n\n\n\n\n            Bramkarz        \n\n',
       'Serbia / Bośnia i Hercegowina', '11 kwi 1998 (25)', '1,92m',
       'prawonożny', '1 lip 2021', '30 cze 2026', 'Raków Częstochowa',
       '8,00 mln €'], dtype='<U110')

# Transposing data

In [19]:
trans = np.transpose(new_2d_arr)

In [20]:
trans[:,0]

array(['\n\n\n\n \n\n\n                Vladan Kovacevic            \n\n\n\n\n            Bramkarz        \n\n',
       '\n\n\n\n \n\n\n                Kacper Trelowski            \n\n\n\n\n            Bramkarz        \n\n',
       '\n\n\n\n \n\n\n                Jakub Rajczykowski            \n\n\n\n\n            Bramkarz        \n\n',
       '\n\n\n\n \n\n\n                Stratos Svarnas            \n\n\n\n\n            Środkowy obrońca        \n\n',
       '\n\n\n\n \n\n\n                Zoran Arsenic            \n\n\n\n\n            Środkowy obrońca        \n\n',
       '\n\n\n\n \n\n\n                Bogdan Racovitan            \n\n\n\n\n            Środkowy obrońca        \n\n',
       '\n\n\n\n \n\n\n                Adrian Gryszkiewicz\xa0 \n\n\n\n\n            Środkowy obrońca        \n\n',
       '\n\n\n\n \n\n\n                Tomas Petrasek            \n\n\n\n\n            Środkowy obrońca        \n\n',
       '\n\n\n\n \n\n\n                Milan Rundic            \n\n\n\n\

# Cleaning a first column

### Creating a list without any spaces and enters "\n"

In [21]:
split = np.char.split(trans[:,0])

In [22]:
split[1]

['Kacper', 'Trelowski', 'Bramkarz']

In [23]:
excluded_positions = ["Bramkarz","Środkowy","obrońca","Ofensywny","pomocnik","Defensywny","napastnik","Lewy","Prawy","Cofnięty"]

In [24]:
list_name = []

### Creating a list of names from the first column

In [25]:
for row in split:
    combined_name = ' '.join([name for name in row if name not in excluded_positions])
    list_name.append(combined_name)

In [26]:
list_name

['Vladan Kovacevic',
 'Kacper Trelowski',
 'Jakub Rajczykowski',
 'Stratos Svarnas',
 'Zoran Arsenic',
 'Bogdan Racovitan',
 'Adrian Gryszkiewicz',
 'Tomas Petrasek',
 'Milan Rundic',
 'Giannis Papanikolaou',
 'Ben Lederman',
 'Gustav Berggren',
 'Szymon Czyz',
 'Tobiasz Kubik',
 'Olivier Sukiennicki',
 'Fran Tudor',
 'Wiktor Dlugosz',
 'Patryk Kun',
 'Jean Carlos Silva',
 'Ivi López',
 'Vladyslav Kochergin',
 'Bartosz Nowak',
 'Mateusz Wdowiak',
 'Marcin Cebula',
 'Vladislavs Gutkovskis',
 'Fabian Piasecki',
 'Sebastian Musiolik',
 'Filip Bednarek',
 'Dominik Holec',
 'Artur Rudko',
 'Filip Dagerstal',
 'Antonio Milic',
 'Lubomir Satka',
 'Bartosz Salamon',
 'Michal Gurgul',
 'Pedro Rebocho',
 'Barry Douglas',
 'Joel Pereira',
 'Mateusz Zukowski',
 'Alan Czerwinski',
 'Jesper Karlström',
 'Nika Kvekveskiri',
 'Radoslaw Murawski',
 'Afonso Sousa',
 'Filip Marchwinski',
 'João Amaral',
 'Maksym Czekala',
 'Michal Skoras',
 'Kristoffer Velde',
 'Jakub Antczak',
 'Georgiy Tsitaishvili',
 

### cleandata array are data that doesn't need any changes

In [27]:
cleandata = trans[:,1:]

In [28]:
df = pd.DataFrame(cleandata)

In [29]:
name = np.chararray.strip(list_name)

In [30]:
name

chararray(['Vladan Kovacevic', 'Kacper Trelowski', 'Jakub Rajczykowski',
           'Stratos Svarnas', 'Zoran Arsenic', 'Bogdan Racovitan',
           'Adrian Gryszkiewicz', 'Tomas Petrasek', 'Milan Rundic',
           'Giannis Papanikolaou', 'Ben Lederman', 'Gustav Berggren',
           'Szymon Czyz', 'Tobiasz Kubik', 'Olivier Sukiennicki',
           'Fran Tudor', 'Wiktor Dlugosz', 'Patryk Kun',
           'Jean Carlos Silva', 'Ivi López', 'Vladyslav Kochergin',
           'Bartosz Nowak', 'Mateusz Wdowiak', 'Marcin Cebula',
           'Vladislavs Gutkovskis', 'Fabian Piasecki',
           'Sebastian Musiolik', 'Filip Bednarek', 'Dominik Holec',
           'Artur Rudko', 'Filip Dagerstal', 'Antonio Milic',
           'Lubomir Satka', 'Bartosz Salamon', 'Michal Gurgul',
           'Pedro Rebocho', 'Barry Douglas', 'Joel Pereira',
           'Mateusz Zukowski', 'Alan Czerwinski', 'Jesper Karlström',
           'Nika Kvekveskiri', 'Radoslaw Murawski', 'Afonso Sousa',
           'Filip M

### Creating list of positions

In [31]:
list_position = [[x for x in row if x in excluded_positions] for row in split]

In [32]:
list_position

[['Bramkarz'],
 ['Bramkarz'],
 ['Bramkarz'],
 ['Środkowy', 'obrońca'],
 ['Środkowy', 'obrońca'],
 ['Środkowy', 'obrońca'],
 ['Środkowy', 'obrońca'],
 ['Środkowy', 'obrońca'],
 ['Środkowy', 'obrońca'],
 ['Defensywny', 'pomocnik'],
 ['Defensywny', 'pomocnik'],
 ['Środkowy', 'pomocnik'],
 ['Środkowy', 'pomocnik'],
 ['Środkowy', 'pomocnik'],
 ['Środkowy', 'pomocnik'],
 ['Prawy', 'pomocnik'],
 ['Prawy', 'pomocnik'],
 ['Lewy', 'pomocnik'],
 ['Lewy', 'pomocnik'],
 ['Ofensywny', 'pomocnik'],
 ['Ofensywny', 'pomocnik'],
 ['Ofensywny', 'pomocnik'],
 ['Ofensywny', 'pomocnik'],
 ['Ofensywny', 'pomocnik'],
 ['Środkowy', 'napastnik'],
 ['Środkowy', 'napastnik'],
 ['Środkowy', 'napastnik'],
 ['Bramkarz'],
 ['Bramkarz'],
 ['Bramkarz'],
 ['Środkowy', 'obrońca'],
 ['Środkowy', 'obrońca'],
 ['Środkowy', 'obrońca'],
 ['Środkowy', 'obrońca'],
 ['Środkowy', 'obrońca'],
 ['Lewy', 'obrońca'],
 ['Lewy', 'obrońca'],
 ['Prawy', 'obrońca'],
 ['Prawy', 'obrońca'],
 ['Prawy', 'obrońca'],
 ['Defensywny', 'pomocnik']

### Unnesting a list of positions

In [33]:
new_list_position = []

for item in list_position:
    if type(item) == list:
        if len(item) == 2:
            new_item = ' '.join(item)
            new_list_position.append(new_item)
        else:
            new_list_position.extend(item)

In [34]:
new_list_position

['Bramkarz',
 'Bramkarz',
 'Bramkarz',
 'Środkowy obrońca',
 'Środkowy obrońca',
 'Środkowy obrońca',
 'Środkowy obrońca',
 'Środkowy obrońca',
 'Środkowy obrońca',
 'Defensywny pomocnik',
 'Defensywny pomocnik',
 'Środkowy pomocnik',
 'Środkowy pomocnik',
 'Środkowy pomocnik',
 'Środkowy pomocnik',
 'Prawy pomocnik',
 'Prawy pomocnik',
 'Lewy pomocnik',
 'Lewy pomocnik',
 'Ofensywny pomocnik',
 'Ofensywny pomocnik',
 'Ofensywny pomocnik',
 'Ofensywny pomocnik',
 'Ofensywny pomocnik',
 'Środkowy napastnik',
 'Środkowy napastnik',
 'Środkowy napastnik',
 'Bramkarz',
 'Bramkarz',
 'Bramkarz',
 'Środkowy obrońca',
 'Środkowy obrońca',
 'Środkowy obrońca',
 'Środkowy obrońca',
 'Środkowy obrońca',
 'Lewy obrońca',
 'Lewy obrońca',
 'Prawy obrońca',
 'Prawy obrońca',
 'Prawy obrońca',
 'Defensywny pomocnik',
 'Defensywny pomocnik',
 'Środkowy pomocnik',
 'Ofensywny pomocnik',
 'Ofensywny pomocnik',
 'Ofensywny pomocnik',
 'Ofensywny pomocnik',
 'Lewy napastnik',
 'Lewy napastnik',
 'Lewy na

## Finalization of transformation data, normalize process and column naming.

In [35]:
new_list_position_pad = np.pad(new_list_position, (0, len(cleandata) - len(new_list_position)), 'constant')


In [36]:
cleaned_data = np.column_stack((name, new_list_position_pad,cleandata))

In [37]:
header_columns = ["Imie i Nazwisko","Pozycja","Narodowość","Rok urodzenia","Wzrost","Lepsza noga","W drużynie od","Kontrakt", "club_id","Wartość rynkowa"]

In [38]:
cleaned_data[:,9] = np.char.replace(cleaned_data[:,9], ",", "")

In [39]:
cleaned_data[:,9] = np.char.replace(cleaned_data[:,9], " mln €", "0000")
cleaned_data[:,9] = np.char.replace(cleaned_data[:,9], " tys. €", "000")

In [40]:
cleaned_data

array([['Vladan Kovacevic', 'Bramkarz', 'Serbia / Bośnia i Hercegowina',
        ..., '30 cze 2026', 'Raków Częstochowa', '8000000'],
       ['Kacper Trelowski', 'Bramkarz', 'Polska', ..., '30 cze 2025',
        'Raków Częstochowa', '700000'],
       ['Jakub Rajczykowski', 'Bramkarz', 'Polska', ..., '30 cze 2025',
        'Raków Częstochowa', '25000'],
       ...,
       ['Ángelo Henríquez', 'Środkowy napastnik', 'Chile', ...,
        '30 cze 2025', 'Miedź Legnica', '350000'],
       ['Wiktor Bogacz', 'Środkowy napastnik', 'Polska', ...,
        '31 gru 2025', 'Miedź Legnica', '25000'],
       ['Hubert Kwolek', '0', 'Polska', ..., '30 cze 2023',
        'Miedź Legnica', '25000']], dtype='<U110')

In [41]:
df = pd.DataFrame(cleaned_data)

In [42]:
df

,0,1,2,3,4,5,6,7,8,9
0,Vladan Kovacevic,Bramkarz,Serbia / Bośnia i Hercegowina,11 kwi 1998 (25),"1,92m",prawonożny,1 lip 2021,30 cze 2026,Raków Częstochowa,8000000
1,Kacper Trelowski,Bramkarz,Polska,19 sie 2003 (19),"1,93m",obunożny,23 sie 2019,30 cze 2025,Raków Częstochowa,700000
2,Jakub Rajczykowski,Bramkarz,Polska,4 lut 2005 (18),-,,21 lip 2022,30 cze 2025,Raków Częstochowa,25000
3,Stratos Svarnas,Środkowy obrońca,Grecja,11 lis 1997 (25),"1,86m",prawonożny,1 lip 2022,30 cze 2023,Raków Częstochowa,2000000
4,Zoran Arsenic,Środkowy obrońca,Chorwacja,2 cze 1994 (29),"1,88m",prawonożny,1 lip 2021,30 cze 2026,Raków Częstochowa,1800000
...,...,...,...,...,...,...,...,...,...,...
546,Igor Lewandowski,Ofensywny pomocnik,Polska,9 paź 2001 (21),"1,76m",,28 wrz 2020,-,Miedź Legnica,25000
547,Blazej Szczepanek,Środkowy napastnik,Polska,20 mar 2001 (22),"1,77m",prawonożny,28 paź 2022,30 cze 2024,Miedź Legnica,-
548,Ángelo Henríquez,Środkowy napastnik,Chile,13 kwi 1994 (29),"1,77m",prawonożny,1 lip 2022,30 cze 2025,Miedź Legnica,350000
549,Wiktor Bogacz,Środkowy napastnik,Polska,14 lip 2004 (18),"1,93m",obunożny,6 kwi 2023,31 gru 2025,Miedź Legnica,25000


In [43]:
club_lookup = np.unique(cleaned_data[:,8])

In [44]:
league_name = [league_name]

In [45]:
league_lookup = pd.DataFrame(league_name)

In [46]:
league_lookup.columns = ['league_name']

In [47]:
league_lookup

,league_name
0,PKO Ekstraklasa


# Creating .csv file
Script checking if a club_data.csv is existing in the same directory as script, if so, then appending data to the same .csv, if doesn't, then creating a new .csv.

### For League_Lookup table

In [48]:
directory_file_csv = os.path.join(os.getcwd(), "League-Lookup.csv")
if os.path.isfile(directory_file_csv):
    exist_league_lookup = pd.read_csv("League-Lookup.csv")
    exist_league_lookup.columns = ['league_id', 'league_name']
    combine_lookup = pd.concat([exist_league_lookup, league_lookup], axis = 0, ignore_index=True)
    combine_lookup = combine_lookup.reset_index()
    combine_lookup = combine_lookup.drop('league_id', axis=1)
    combine_lookup.columns = ['league_id', 'league_name']
    combine_lookup.to_csv("League-Lookup.csv", index = False)
    new_index = combine_lookup.iloc[-1]['league_id']
    foreign_key = [new_index] * len(club_lookup)
    club_lookup = np.column_stack((club_lookup, foreign_key))
else:
    league_lookup = league_lookup.reset_index()
    foreign_key = [0] * len(club_lookup)
    club_lookup = np.column_stack((club_lookup, foreign_key))
    league_lookup.columns = ['league_id','league_name']
    league_lookup.to_csv("League-Lookup.csv", index = False)

### For Club_Lookup table

In [49]:
club_lookup = pd.DataFrame(club_lookup)

In [50]:
club_lookup.columns = ['club_name','league_id']

In [51]:
directory_file_csv = os.path.join(os.getcwd(), "Club-Lookup.csv")
if os.path.isfile(directory_file_csv):
    exist_club_lookup = pd.read_csv("Club-Lookup.csv")
    exist_club_lookup.columns = ['club_id', 'club_name','league_id']
    combine_lookup = pd.concat([exist_club_lookup, club_lookup], axis = 0, ignore_index=True)
    combine_lookup = combine_lookup.reset_index()
    combine_lookup = combine_lookup.drop('club_id', axis=1)
    combine_lookup.columns = ['club_id', 'club_name','league_id']
    combine_lookup.to_csv("Club-Lookup.csv", index = False)
    amount_from_end = combine_lookup.iloc[-len(club_lookup)]['club_id']
    new_index = combine_lookup.iloc[amount_from_end:]
    map_dict = dict(zip(new_index['club_name'], new_index['club_id']))
    df[8] = df[8].map(map_dict)
    
else:
    club_lookup = club_lookup.reset_index()
    map_dict = dict(zip(club_lookup['club_name'], club_lookup['index']))
    df[8] = df[8].map(map_dict)
    club_lookup.columns = ['club_id','club_name','league_id']
    club_lookup.to_csv("Club-Lookup.csv", index = False)

### For Fact table

In [52]:
directory_file_csv = os.path.join(os.getcwd(), "League_Player_Data.csv")

if os.path.isfile(directory_file_csv):
    df.to_csv('League_Player_Data.csv', mode='a', index=False, header=False)
else:
    df.columns = header_columns
    df.to_csv("League_Player_Data.csv", index = False)

In [53]:
#remove = pd.read_csv('club_data.csv')
#remove = remove.drop(remove.index[93:])
#remove.to_csv('club_data.csv')
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [54]:
data = pd.read_csv("League_Player_Data.csv")

In [55]:
data

,Imie i Nazwisko,Pozycja,Narodowość,Rok urodzenia,Wzrost,Lepsza noga,W drużynie od,Kontrakt,club_id,Wartość rynkowa
0,Manuel Neuer,Bramkarz,Niemcy,27 mar 1986 (37),"1,93m",prawonożny,1 lip 2011,30 cze 2024,4,7000000
1,Yann Sommer,Bramkarz,Szwajcaria,17 gru 1988 (34),"1,83m",prawonożny,19 sty 2023,30 cze 2025,4,5000000
2,Sven Ulreich,Bramkarz,Niemcy,3 sie 1988 (34),"1,92m",prawonożny,1 lip 2021,30 cze 2024,4,700000
3,Johannes Schenk,Bramkarz,Niemcy,13 sty 2003 (20),"1,91m",,1 lip 2022,30 cze 2024,4,300000
4,Matthijs de Ligt,Środkowy obrońca,Holandia,12 sie 1999 (23),"1,89m",prawonożny,19 lip 2022,30 cze 2027,4,75000000
...,...,...,...,...,...,...,...,...,...,...
3720,Igor Lewandowski,Ofensywny pomocnik,Polska,9 paź 2001 (21),"1,76m",,28 wrz 2020,-,122,25000
3721,Blazej Szczepanek,Środkowy napastnik,Polska,20 mar 2001 (22),"1,77m",prawonożny,28 paź 2022,30 cze 2024,122,-
3722,Ángelo Henríquez,Środkowy napastnik,Chile,13 kwi 1994 (29),"1,77m",prawonożny,1 lip 2022,30 cze 2025,122,350000
3723,Wiktor Bogacz,Środkowy napastnik,Polska,14 lip 2004 (18),"1,93m",obunożny,6 kwi 2023,31 gru 2025,122,25000


In [56]:
lookup_c = pd.read_csv("Club-Lookup.csv")
lookup_l = pd.read_csv("League-Lookup.csv")
lookup_c

,club_id,club_name,league_id
0,0,1.FC Köln,0
1,1,1.FC Union Berlin,0
2,2,1.FSV Mainz 05,0
3,3,Bayer 04 Leverkusen,0
4,4,Bayern Monachium,0
...,...,...,...
128,128,Warta Poznań,6
129,129,Widzew Łódź,6
130,130,Wisła Płock,6
131,131,Zagłębie Lubin,6


In [57]:
lookup_l

,league_id,league_name
0,0,Bundesliga
1,1,Serie A
2,2,Ligue 1
3,3,Premier League
4,4,LaLiga
5,5,Eredivisie
6,6,PKO Ekstraklasa
